Imports


In [ ]:
import torch
import torchvision
import torch.nn as nn
from tqdm import tqdm
import multiprocessing
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

Device:  cuda


Transformacions


In [ ]:
  train_transform = transforms.Compose(
      [
          transforms.RandomRotation(15),
          transforms.RandomAffine(
              degrees=2, translate=(0.002, 0.001), scale=(0.001, 1.64)
          ),
          transforms.RandomResizedCrop(28, scale=(0.8, 1.0)),
          transforms.RandomPerspective(distortion_scale=0.5, p=0.5),

          transforms.ToTensor(),
      ]
  )

  test_transform = transforms.Compose(
      [
          transforms.ToTensor(),
      ]
  )

Dataset Class


In [ ]:
class MNIST_dataset(Dataset):
    def __init__(self, partition="train", transform=None):
        print("\nLoading MNIST ", partition, " Dataset...")
        self.partition = partition
        self.transform = transform
        if self.partition == "train":
            self.data = torchvision.datasets.MNIST(".data/", train=True, download=True)
        else:
            self.data = torchvision.datasets.MNIST(".data/", train=False, download=True)
        print("\tTotal Len.: ", len(self.data), "\n", 50 * "-")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Image
        image = self.data[idx][0]
        image = self.transform(image)
        # care! net expect a 784 size vector and our dataset
        # provide 1x28x28 (channels, height, width) -> Reshape!
        image = image.view(-1)

        # Label
        label = torch.tensor(self.data[idx][1])
        label = F.one_hot(label, num_classes=10).float()

        return {"idx": idx, "img": image, "label": label}

Classe Net


In [ ]:
class Net(nn.Module):
    def __init__(
        self,
        sizes=[[784, 1024], [1024, 1024], [1024, 1024], [1024, 10]],
        criterion=None,
    ):
        super(Net, self).__init__()


        self.linear1 = nn.Linear(784, 1024)
        self.batch_norm1 = nn.BatchNorm1d(1024)
        self.relu1 = F.gelu
        self.linear2 = nn.Linear(1024, 1024)
        self.batch_norm2 = nn.BatchNorm1d(1024)
        self.relu2 = F.gelu
        self.linear3 = nn.Linear(1024, 1024)
        self.batch_norm3 = nn.BatchNorm1d(1024)
        self.relu3 = F.gelu
        self.classifier = nn.Linear(1024, num_classes)
        self.criterion = criterion

    def forward(self, x, y=None):
        x = self.relu1(self.batch_norm1(self.linear1(x)))
        x = self.relu2(self.batch_norm2(self.linear2(x)))
        x = self.relu3(self.batch_norm3(self.linear3(x)))
        x = self.classifier(x)

        if y != None:
            loss = self.criterion(x, y)
            return loss, x
        return x

        if y != None:
            loss = self.criterion(x, labels)
            return loss, x
        return x

Preparació datasets


In [ ]:
train_dataset = MNIST_dataset(partition="train", transform=train_transform)
test_dataset = MNIST_dataset(partition="test", transform=test_transform)

batch_size = 100
num_workers = 4
print("Num workers:", num_workers)
train_dataloader = DataLoader(
    train_dataset, batch_size, shuffle=True, num_workers=num_workers
)
test_dataloader = DataLoader(
    test_dataset, batch_size, shuffle=False, num_workers=num_workers
)


Loading MNIST  train  Dataset...
Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 9.91M/9.91M [00:00<00:00, 15.9MB/s]


Extracting .data/MNIST/raw/train-images-idx3-ubyte.gz to .data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 28.9k/28.9k [00:00<00:00, 493kB/s]


Extracting .data/MNIST/raw/train-labels-idx1-ubyte.gz to .data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 1.65M/1.65M [00:00<00:00, 3.85MB/s]


Extracting .data/MNIST/raw/t10k-images-idx3-ubyte.gz to .data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 4.54k/4.54k [00:00<00:00, 2.56MB/s]

Extracting .data/MNIST/raw/t10k-labels-idx1-ubyte.gz to .data/MNIST/raw

	Total Len.:  60000 
 --------------------------------------------------

Loading MNIST  test  Dataset...
	Total Len.:  10000 
 --------------------------------------------------
Num workers: 4



/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Preparació de la xarxa


In [ ]:
# Training Settings
criterion = nn.CrossEntropyLoss()

# Instantiating the network and printing its architecture
num_classes = 10
net = Net(
    sizes=[[784, 1024], [1024, 1024], [1024, 1024], [1024, num_classes]],
    criterion=criterion,
)
print(net)
optimizer = optim.Adam(net.parameters(), lr=0.01, weight_decay=1e-6)
net = net.to(device)

Net(
  (linear1): Linear(in_features=784, out_features=1024, bias=True)
  (batch_norm1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=1024, out_features=1024, bias=True)
  (batch_norm2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=1024, out_features=1024, bias=True)
  (batch_norm3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (classifier): Linear(in_features=1024, out_features=10, bias=True)
  (criterion): CrossEntropyLoss()
)


Scheduler (Learning rate annealing)


In [ ]:
# Learning Rate Annealing (LRA) scheduling
# lr = 0.1     if epoch < 25
# lr = 0.01    if 25 <= epoch < 50
# lr = 0.001   if epoch >= 50
scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[25, 50], gamma=0.1
)

Entrenament


In [ ]:
# Start training
epochs = 75

print("\n---- Start Training ----")
best_accuracy = -1
best_epoch = 0
for epoch in range(epochs):
    # TRAIN NETWORK
    train_loss, train_correct = 0, 0
    net.train()
    with tqdm(
        iter(train_dataloader), desc="Epoch " + str(epoch), unit="batch"
    ) as tepoch:
        for batch in tepoch:
            images = batch["img"].to(device)
            labels = batch["label"].to(device)
            ids = batch["idx"].to("cpu").numpy()

            # zero the parameter gradients
            optimizer.zero_grad()

            #  Forward
            loss, outputs = net(images, labels)

            loss.backward()

            optimizer.step()

            # one hot -> labels
            labels = torch.argmax(labels, dim=1)
            pred = torch.argmax(outputs, dim=1)

            train_correct += pred.eq(labels).sum().item()

            # print statistics
            train_loss += loss.item()

        scheduler.step()
        print("\tLR: ", optimizer.param_groups[0]["lr"])

    train_loss /= len(train_dataloader.dataset)

    # TEST NETWORK
    test_loss, test_correct = 0, 0
    net.eval()
    with torch.no_grad():
        with tqdm(
            iter(test_dataloader), desc="Test " + str(epoch), unit="batch"
        ) as tepoch:
            for batch in tepoch:
                images = batch["img"].to(device)
                labels = batch["label"].to(device)
                ids = batch["idx"].to("cpu").numpy()

                #  Forward
                outputs = net(images)
                test_loss += criterion(outputs, labels)

                # one hot -> labels
                labels = torch.argmax(labels, dim=1)
                pred = torch.argmax(outputs, dim=1)

                test_correct += pred.eq(labels).sum().item()

    test_loss /= len(test_dataloader.dataset)
    test_accuracy = 100.0 * test_correct / len(test_dataloader.dataset)

    print(
        "[Epoch {}] Train Loss: {:.6f} - Test Loss: {:.6f} - Train Accuracy: {:.2f}% - Test Accuracy: {:.2f}%".format(
            epoch + 1,
            train_loss,
            test_loss,
            100.0 * train_correct / len(train_dataloader.dataset),
            test_accuracy,
        )
    )

    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        best_epoch = epoch

        # Save best weights
        torch.save(net.state_dict(), "best_model.pt")

print("\nBEST TEST ACCURACY: ", best_accuracy, " in epoch ", best_epoch)


---- Start Training ----


Epoch 0: 100%|██████████| 600/600 [00:55<00:00, 10.76batch/s]

	LR:  0.01



Test 0: 100%|██████████| 100/100 [00:02<00:00, 48.24batch/s]

[Epoch 1] Train Loss: 0.013800 - Test Loss: 0.002146 - Train Accuracy: 52.99% - Test Accuracy: 93.44%


...

[Epoch 74] Train Loss: 0.005257 - Test Loss: 0.000226 - Train Accuracy: 81.41% - Test Accuracy: 99.14%



Epoch 74: 100%|██████████| 600/600 [00:41<00:00, 14.46batch/s]

	LR:  0.0001



Test 74: 100%|██████████| 100/100 [00:02<00:00, 45.29batch/s]

[Epoch 75] Train Loss: 0.005280 - Test Loss: 0.000224 - Train Accuracy: 81.42% - Test Accuracy: 99.23%

BEST TEST ACCURACY:  99.3  in epoch  59


Avaluacio


In [ ]:
# Load best weights
net.load_state_dict(torch.load("best_model.pt"))

test_loss, test_correct = 0, 0
net.eval()
with torch.no_grad():
    with tqdm(iter(test_dataloader), desc="Test " + str(epoch), unit="batch") as tepoch:
        for batch in tepoch:
            images = batch["img"].to(device)
            labels = batch["label"].to(device)
            ids = batch["idx"].to("cpu").numpy()

            #  Forward
            outputs = net(images)
            test_loss += criterion(outputs, labels)

            # one hot -> labels
            labels = torch.argmax(labels, dim=1)
            pred = torch.argmax(outputs, dim=1)

            test_correct += pred.eq(labels).sum().item()

    test_loss /= len(test_dataloader.dataset)
    test_accuracy = 100.0 * test_correct / len(test_dataloader.dataset)
print("Final best acc: ", test_accuracy)

<ipython-input-10-add965e72a74>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("best_model.pt"))
Test 74: 100%|██████████| 100/100 [00:02<00

Final best acc:  99.3
